In [ ]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [ ]:
import pandas as pd

class VaersDescrReader:
    
    def __init__(self, dataDir):
        self.dataDir = dataDir        

    def readVaersDescrsForYears(self, years):
        return [self.readVaersDescrForYear(year) for year in years]

    def readVaersDescrForYear(self, year):
        return {
                    'VAERSDATA': self._readVAERSDATA('{dataDir}/{year}VAERSDATA.csv'.format(dataDir = self.dataDir, year = year)),
                    'VAERSVAX': self._readVAERSVAX('{dataDir}/{year}VAERSVAX.csv'.format(dataDir = self.dataDir, year = year))
               }

    def readNonDomesticVaersDescr(self):
        return {
                    'VAERSDATA': self._readVAERSDATA(self.dataDir + "/NonDomesticVAERSDATA.csv"),
                    'VAERSVAX': self._readVAERSVAX(self.dataDir + "/NonDomesticVAERSVAX.csv")
               }

    def _readVAERSDATA(self, file):
        return self._read_csv(
            file = file,
            usecols = ['VAERS_ID', 'RECVDATE', 'DIED', 'L_THREAT', 'DISABLE', 'HOSPITAL', 'ER_VISIT', 'SPLTTYPE'],
            parse_dates = ['RECVDATE'],
            date_parser = lambda dateStr: pd.to_datetime(dateStr, format = "%m/%d/%Y"))

    def _readVAERSVAX(self, file):
        return self._read_csv(
            file = file,
            usecols = ['VAERS_ID', 'VAX_DOSE_SERIES', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT'],
            dtype = {"VAX_DOSE_SERIES": "string"})

    def _read_csv(self, file, **kwargs):
        return pd.read_csv(
            file,
            index_col = 'VAERS_ID',
            encoding = 'latin1',
            low_memory = False,
            **kwargs)


In [ ]:
import pandas as pd

class VaersDescr2DataFrameConverter:

    @staticmethod
    def createDataFrameFromDescr(vaersDescr):
        return pd.merge(
                vaersDescr['VAERSDATA'],
                vaersDescr['VAERSVAX'],
                how = 'left',
                left_index = True,
                right_index = True,
                validate = 'one_to_many')

    @staticmethod
    def createDataFrameFromDescrs(vaersDescrs):
        dataFrames = [VaersDescr2DataFrameConverter.createDataFrameFromDescr(vaersDescr) for vaersDescr in vaersDescrs]
        return pd.concat(dataFrames)


In [ ]:
class DataFrameNormalizer:
    
    @staticmethod
    def normalize(dataFrame):
        DataFrameNormalizer.convertVAX_LOTColumnToUpperCase(dataFrame)
        DataFrameNormalizer._convertColumnsOfDataFrame_Y_to_1_else_0(
            dataFrame,
            ['DIED', 'L_THREAT', 'DISABLE', 'HOSPITAL', 'ER_VISIT'])

    @staticmethod
    def convertVAX_LOTColumnToUpperCase(dataFrame):
        dataFrame['VAX_LOT'] = dataFrame['VAX_LOT'].str.upper()

    @staticmethod
    def _convertColumnsOfDataFrame_Y_to_1_else_0(dataFrame, columns):
        for column in columns:
            DataFrameNormalizer._convertColumnOfDataFrame_Y_to_1_else_0(dataFrame, column)

    @staticmethod
    def _convertColumnOfDataFrame_Y_to_1_else_0(dataFrame, column):
        dataFrame[column] = DataFrameNormalizer._where(
            condition = dataFrame[column] == 'Y',
            trueValue = 1,
            falseValue = 0)

    @staticmethod
    def _where(condition, trueValue, falseValue):
        return np.where(condition, trueValue, falseValue)    
    

In [ ]:
import pandas as pd

class DataFrameFilter:
    
    def filterByCovid19(self, dataFrame):
        return dataFrame[self._isCovid19(dataFrame)]

    def _isCovid19(self, dataFrame):
        return dataFrame["VAX_TYPE"] == "COVID19"


In [ ]:
class SummationTableFactory:

    @staticmethod
    def createSummationTable(
        groupBy,
        columnNameMappingsDict = {
                "DIED_size": "Adverse Reaction Reports",
                "DIED_sum": "Deaths",
                "L_THREAT_sum": "Life Threatening Illnesses",
                "DISABLE_sum": "Disabilities",
                'HOSPITAL_sum': 'Hospitalisations',
                'ER_VISIT_sum': 'Emergency Room or Doctor Visits'
            }):

        summationTable = groupBy.agg({
                'DIED': ['sum', 'size'],
                'L_THREAT': 'sum',
                'DISABLE': 'sum',
                'HOSPITAL': 'sum',
                'ER_VISIT': 'sum',
                'SEVERE': 'sum'
            })
        SummationTableFactory._flattenColumns(summationTable)
        return summationTable.rename(columns = columnNameMappingsDict)

    @staticmethod
    def createSummationTableHavingSevereReportsColumn(dataFrame):
        summationTable = SummationTableFactory.createSummationTable(
            dataFrame,
            columnNameMappingsDict = {
                "DIED_size": "Adverse Reaction Reports",
                "DIED_sum": "Deaths",
                "L_THREAT_sum": "Life Threatening Illnesses",
                "DISABLE_sum": "Disabilities",
                "SEVERE_sum": "Severities"
            })
        summationTable['Severe reports'] = summationTable['Severities'] / summationTable['Adverse Reaction Reports'] * 100
        summationTable['Lethality'] = summationTable['Deaths'] / summationTable['Adverse Reaction Reports'] * 100
        summationTable = summationTable[
            [
                'Adverse Reaction Reports',
                'Deaths',
                'Disabilities',
                'Life Threatening Illnesses',
                'Severe reports',
                'Lethality'
            ]]
        return summationTable

    @staticmethod
    def _flattenColumns(dataFrame):
        dataFrame.columns = ["_".join(a) for a in dataFrame.columns.to_flat_index()]


In [ ]:
import pycountry

class CountryColumnAdder:
    
    @staticmethod
    def addCountryColumn(dataFrame):
        dataFrame['COUNTRY'] = CountryColumnAdder.getCountryColumn(dataFrame)
        return dataFrame.astype({'COUNTRY': "string"})

    @staticmethod
    def getCountryColumn(dataFrame):
        return dataFrame.apply(
            lambda row:
                CountryColumnAdder._getCountryNameOfSplttypeOrDefault(
                 splttype = row['SPLTTYPE'],
                 default = 'Unknown Country'),
            axis = 'columns')

    @staticmethod
    def _getCountryNameOfSplttypeOrDefault(splttype, default):
        if not isinstance(splttype, str):
            return default
        
        country = pycountry.countries.get(alpha_2 = splttype[:2])
        return country.name if country is not None else default

In [ ]:
import pycountry

class SevereColumnAdder:
    
    @staticmethod
    def addSevereColumn(dataFrame):
        dataFrame['SEVERE'] = (dataFrame['DIED'] + dataFrame['L_THREAT'] + dataFrame['DISABLE']) > 0
        dataFrame['SEVERE'].replace({True: 1, False: 0}, inplace = True)
        return dataFrame


In [ ]:
class CompanyColumnAdder:
    
    @staticmethod
    def addCompanyColumn(batchCodeTable, companyByBatchCodeTable):
        return pd.merge(
            batchCodeTable,
            companyByBatchCodeTable,
            how = 'left',
            left_index = True,
            right_index = True,
            validate = 'one_to_one')

    @staticmethod
    def createCompanyByBatchCodeTable(dataFrame):
        return CompanyColumnAdder._createManufacturerByBatchCodeTable(dataFrame).rename(columns = {"VAX_MANU": "Company"})

    @staticmethod
    def _createManufacturerByBatchCodeTable(dataFrame):
        manufacturerByBatchCodeTable = dataFrame[['VAX_LOT', 'VAX_MANU']]
        manufacturerByBatchCodeTable = manufacturerByBatchCodeTable.drop_duplicates(subset = ['VAX_LOT'])
        return manufacturerByBatchCodeTable.set_index('VAX_LOT')


In [ ]:
class InternationalLotTableFactory:
    
    def __init__(self, dataFrame : pd.DataFrame):
        self.dataFrame = DataFrameFilter().filterByCovid19(dataFrame)
        self.batchCodeTableByCountryFactory = BatchCodeTableByCountryFactory(dataFrame)

    def createInternationalLotTable(self):
        internationalLotTable = self._createInternationalLotTable()
        return internationalLotTable.sort_values(by = 'Severe reports', ascending = False)

    def createBatchCodeTableByCountry(self, country):
        return self.batchCodeTableByCountryFactory.createBatchCodeTableByCountry(country)

    def createGlobalBatchCodeTable(self):
        return self.createBatchCodeTableByCountry(None)

    def _createInternationalLotTable(self):
        return SummationTableFactory.createSummationTableHavingSevereReportsColumn(self.dataFrame.groupby(self.dataFrame['COUNTRY']))


In [ ]:
class BatchCodeTableByCountryFactory:

    def __init__(self, dataFrame : pd.DataFrame):
        self.dataFrame = DataFrameFilter().filterByCovid19(dataFrame)
        self.countryBatchCodeTable = None

    def createBatchCodeTableByCountry(self, country):
        batchCodeTable = self._createBatchCodeTableByCountry(country)
        batchCodeTable = CompanyColumnAdder.addCompanyColumn(batchCodeTable, CompanyColumnAdder.createCompanyByBatchCodeTable(self.dataFrame))
        batchCodeTable = batchCodeTable[
            [
                'Adverse Reaction Reports',
                'Deaths',
                'Disabilities',
                'Life Threatening Illnesses',
                'Company',
                'Severe reports',
                'Lethality'
            ]]
        return batchCodeTable.sort_values(by = 'Severe reports', ascending = False)

    # FK-TODO: refactor
    def _createBatchCodeTableByCountry(self, country):
        if country is None:
            return SummationTableFactory.createSummationTableHavingSevereReportsColumn(self.dataFrame.groupby('VAX_LOT'))

        if self.countryBatchCodeTable is None:
            self.countryBatchCodeTable = self._getCountryBatchCodeTable()
            
        return self._getCountry(self.countryBatchCodeTable, country)

    def _getCountryBatchCodeTable(self):
        return SummationTableFactory.createSummationTableHavingSevereReportsColumn(
            self.dataFrame.groupby(
                [
                    self.dataFrame['COUNTRY'],
                    self.dataFrame['VAX_LOT']
                ]))

    def _getCountry(self, countryBatchCodeTable, country):
        return countryBatchCodeTable.loc[country] if country in countryBatchCodeTable.index else self._getEmptyBatchCodeTable(countryBatchCodeTable)
    
    def _getEmptyBatchCodeTable(self, countryBatchCodeTable):
        return countryBatchCodeTable[0:0].droplevel(0)

    

In [ ]:
import os

class IOUtils:

    @staticmethod
    def saveDataFrame(dataFrame, file):
        IOUtils.saveDataFrameAsExcelFile(dataFrame, file)
        IOUtils.saveDataFrameAsHtml(dataFrame, file)
        IOUtils.saveDataFrameAsJson(dataFrame, file)

    @staticmethod
    def saveDataFrameAsExcelFile(dataFrame, file):
        IOUtils.ensurePath(file)
        dataFrame.to_excel(file + '.xlsx')

    @staticmethod
    def saveDataFrameAsHtml(dataFrame, file):
        IOUtils.ensurePath(file)
        dataFrame.reset_index().to_html(
            file + '.html',
            index = False,
            table_id = 'batchCodeTable',
            classes = 'display',
            justify = 'unset',
            border = 0)

    @staticmethod
    def saveDataFrameAsJson(dataFrame, file):
        IOUtils.ensurePath(file)
        dataFrame.reset_index().to_json(
            file + '.json',
            orient = "split",
            index = False)

    @staticmethod
    def ensurePath(file):
        directory = os.path.dirname(file)
        if not os.path.exists(directory):
            os.makedirs(directory)


In [ ]:
import unittest

In [ ]:
class TestHelper:

    @staticmethod
    def createDataFrame(index, columns, data, dtypes = {}):
        return pd.DataFrame(index = index, columns = columns, data = data).astype(dtypes)


In [ ]:
from pandas.testing import assert_frame_equal

class DataFrameNormalizerTest(unittest.TestCase):

    def test_convertVAX_LOTColumnToUpperCase(self):
        # Given
        dataFrame = TestHelper.createDataFrame(
            columns = ['VAX_LOT'],
            data = [  ['037K20A'],
                      ['025l20A'],
                      ['025L20A']],
            index = [
                "0916600",
                "0916601",
                "1996874"])
            
        # When
        DataFrameNormalizer.convertVAX_LOTColumnToUpperCase(dataFrame)
        
        # Then
        dataFrameExpected = TestHelper.createDataFrame(
            columns = ['VAX_LOT'],
            data = [  ['037K20A'],
                      ['025L20A'],
                      ['025L20A']],
            index = [
                "0916600",
                "0916601",
                "1996874"])
        assert_frame_equal(dataFrame, dataFrameExpected, check_dtype = False)


In [ ]:
from pandas.testing import assert_frame_equal

class DataFrameFilterTest(unittest.TestCase):

    def test_filterByCovid19(self):
        # Given
        dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(
            [
                {
                    'VAERSDATA': TestHelper.createDataFrame(
                        columns = ['DIED', 'L_THREAT', 'DISABLE'],
                        data = [  [1,      0,          0],
                                  [0,      0,          1]],
                        index = [
                            "0916600",
                            "0916601"]),
                    'VAERSVAX': TestHelper.createDataFrame(
                        columns = ['VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES'],
                        data = [  ['COVID19',  'MODERNA',  '037K20A', '1'],
                                  ['COVID19',  'MODERNA',  '025L20A', '1']],
                        index = [
                            "0916600",
                            "0916601"],
                        dtypes = {'VAX_DOSE_SERIES': "string"})
                },
                {
                    'VAERSDATA': TestHelper.createDataFrame(
                        columns = ['DIED', 'L_THREAT', 'DISABLE'],
                        data = [  [0,       0,         0],
                                  [0,       0,         1]],
                            index = [
                            "1996873",
                            "1996874"]),
                    'VAERSVAX': TestHelper.createDataFrame(
                            columns = ['VAX_TYPE', 'VAX_MANU',         'VAX_LOT', 'VAX_DOSE_SERIES'],
                            data = [  ['HPV9',     'MERCK & CO. INC.', 'R017624', 'UNK'],
                                      ['COVID19',  'MODERNA',          '025L20A', '1']],
                            index = [
                                "1996873",
                                "1996874"],
                            dtypes = {'VAX_DOSE_SERIES': "string"})
                    }
            ])
        dataFrameFilter = DataFrameFilter()
            
        # When
        dataFrame = dataFrameFilter.filterByCovid19(dataFrame)
        
        # Then
        dataFrameExpected = TestHelper.createDataFrame(
            columns = ['DIED', 'L_THREAT', 'DISABLE', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES'],
            data = [  [1,       0,         0,         'COVID19',  'MODERNA',  '037K20A', '1'],
                      [0,       0,         1,         'COVID19',  'MODERNA',  '025L20A', '1'],
                      [0,       0,         1,         'COVID19',  'MODERNA',  '025L20A', '1']],
            index = [
                "0916600",
                "0916601",
                "1996874"],
            dtypes = {'VAX_DOSE_SERIES': "string"})
        assert_frame_equal(dataFrame, dataFrameExpected, check_dtype = False)


In [ ]:
from pandas.testing import assert_frame_equal

class InternationalLotTableFactoryTest(unittest.TestCase):

    def test_createInternationalLotTable(self):
        # Given
        dataFrame = TestHelper.createDataFrame(
            columns = ['DIED', 'L_THREAT', 'DISABLE', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES', 'SPLTTYPE',                  'HOSPITAL', 'ER_VISIT', 'COUNTRY'],
            data = [  [1,      0,          0,         'COVID19',  'MODERNA',  '016M20A', '2',               'GBPFIZER INC2020486806',    0,          0,          'United Kingdom'],
                      [1,      0,          0,         'COVID19',  'MODERNA',  '030L20A', '1',               'FRMODERNATX, INC.MOD20224', 0,          0,          'France'],
                      [1,      1,          1,         'COVID19',  'MODERNA',  '030L20B', '1',               'FRMODERNATX, INC.MOD20224', 0,          0,          'France'],
                      [0,      0,          0,         'COVID19',  'MODERNA',  '030L20B', '1',               'dummy',                     0,          0,          'Unknown Country'],
                      [0,      0,          0,         'COVID19',  'MODERNA',  '030L20B', '1',               123,                         0,          0,          'Unknown Country']],
            index = [
                "1048786",
                "1048786",
                "4711",
                "0815",
                "0816"])
        dataFrame = SevereColumnAdder.addSevereColumn(dataFrame)
        internationalLotTableFactory = InternationalLotTableFactory(dataFrame)
        
        # When
        internationalLotTable = internationalLotTableFactory.createInternationalLotTable()

        # Then
        assert_frame_equal(
            internationalLotTable,
            TestHelper.createDataFrame(
                columns = ['Adverse Reaction Reports', 'Deaths', 'Disabilities', 'Life Threatening Illnesses', 'Severe reports', 'Lethality'],
                data = [  [2,                          2,        1,              1,                            2/2 * 100,        2/2 * 100],
                          [1,                          1,        0,              0,                            1/1 * 100,        1/1 * 100],
                          [2,                          0,        0,              0,                            0/2 * 100,        0/2 * 100]],
                index = pd.Index(
                    [
                        'France',
                        'United Kingdom',
                        'Unknown Country'
                    ],
                    name = 'COUNTRY')))

    def test_createBatchCodeTableByCountry(self):
        # Given
        dataFrame = TestHelper.createDataFrame(
            columns = ['DIED', 'L_THREAT', 'DISABLE', 'VAX_TYPE', 'VAX_MANU',         'VAX_LOT', 'VAX_DOSE_SERIES', 'SPLTTYPE',                  'HOSPITAL', 'ER_VISIT', 'COUNTRY'],
            data = [  [1,      0,          0,         'COVID19',  'PFIZER\BIONTECH',  '016M20A', '2',               'GBPFIZER INC2020486806',    0,          0,          'United Kingdom'],
                      [0,      0,          0,         'COVID19',  'MODERNA',          '030L20A', '1',               'FRMODERNATX, INC.MOD20224', 0,          0,          'France'],
                      [1,      1,          1,         'COVID19',  'MODERNA',          '030L20B', '1',               'FRMODERNATX, INC.MOD20224', 0,          0,          'France'],
                      [0,      1,          1,         'COVID19',  'MODERNA',          '030L20B', '1',               'FRMODERNATX, INC.MOD20224', 0,          0,          'France']],
            index = [
                "1048786",
                "1048786",
                "4711",
                "0815"])
        dataFrame = SevereColumnAdder.addSevereColumn(dataFrame)
        internationalLotTableFactory = InternationalLotTableFactory(dataFrame)
        
        # When
        batchCodeTable = internationalLotTableFactory.createBatchCodeTableByCountry('France')

        # Then
        assert_frame_equal(
            batchCodeTable,
            TestHelper.createDataFrame(
                columns = ['Adverse Reaction Reports', 'Deaths', 'Disabilities', 'Life Threatening Illnesses', 'Company', 'Severe reports', 'Lethality'],
                data = [  [2,                          1,        2,              2,                            'MODERNA', 2/2 * 100,        1/2 * 100],
                          [1,                          0,        0,              0,                            'MODERNA', 0/1 * 100,        0/1 * 100]],
                index = pd.Index(
                    [
                        '030L20B',
                        '030L20A'
                    ],
                    name = 'VAX_LOT')),
            check_dtype = False)

    def test_createGlobalBatchCodeTable(self):
        # Given
        dataFrame = TestHelper.createDataFrame(
            columns = ['DIED', 'L_THREAT', 'DISABLE', 'VAX_TYPE', 'VAX_MANU',         'VAX_LOT', 'VAX_DOSE_SERIES', 'SPLTTYPE',                  'HOSPITAL', 'ER_VISIT', 'COUNTRY'],
            data = [  [1,      0,          0,         'COVID19',  'PFIZER\BIONTECH',  '016M20A', '2',               'GBPFIZER INC2020486806',    0,          0,          'United Kingdom'],
                      [0,      0,          0,         'COVID19',  'MODERNA',          '030L20A', '1',               'FRMODERNATX, INC.MOD20224', 0,          0,          'France'],
                      [1,      1,          1,         'COVID19',  'MODERNA',          '030L20B', '1',               'FRMODERNATX, INC.MOD20224', 0,          0,          'France'],
                      [0,      1,          1,         'COVID19',  'MODERNA',          '030L20B', '1',               'FRMODERNATX, INC.MOD20224', 0,          0,          'France']],
            index = [
                "1048786",
                "1048786",
                "4711",
                "0815"])
        dataFrame = SevereColumnAdder.addSevereColumn(dataFrame)
        internationalLotTableFactory = InternationalLotTableFactory(dataFrame)
        
        # When
        batchCodeTable = internationalLotTableFactory.createGlobalBatchCodeTable()

        # Then
        assert_frame_equal(
            batchCodeTable,
            TestHelper.createDataFrame(
                columns = ['Adverse Reaction Reports', 'Deaths', 'Disabilities', 'Life Threatening Illnesses', 'Company',         'Severe reports', 'Lethality'],
                data = [  [1,                          1,        0,              0,                            'PFIZER\BIONTECH', 1/1 * 100,        1/1 * 100],
                          [2,                          1,        2,              2,                            'MODERNA',         2/2 * 100,        1/2 * 100],
                          [1,                          0,        0,              0,                            'MODERNA',         0/1 * 100,        0/1 * 100]],
                index = pd.Index(
                    [
                        '016M20A',
                        '030L20B',
                        '030L20A'
                    ],
                    name = 'VAX_LOT')),
            check_dtype = False)

    def test_createBatchCodeTableByNonExistingCountry(self):
        # Given
        dataFrame = TestHelper.createDataFrame(
            columns = ['DIED', 'L_THREAT', 'DISABLE', 'VAX_TYPE', 'VAX_MANU',         'VAX_LOT', 'VAX_DOSE_SERIES', 'SPLTTYPE',                  'HOSPITAL', 'ER_VISIT', 'COUNTRY'],
            data = [  [1,      0,          0,         'COVID19',  'PFIZER\BIONTECH',  '016M20A', '2',               'GBPFIZER INC2020486806',    0,          0,          'United Kingdom'],
                      [0,      0,          0,         'COVID19',  'MODERNA',          '030L20A', '1',               'FRMODERNATX, INC.MOD20224', 0,          0,          'France'],
                      [1,      1,          1,         'COVID19',  'MODERNA',          '030L20B', '1',               'FRMODERNATX, INC.MOD20224', 0,          0,          'France'],
                      [0,      1,          1,         'COVID19',  'MODERNA',          '030L20B', '1',               'FRMODERNATX, INC.MOD20224', 0,          0,          'France']],
            index = [
                "1048786",
                "1048786",
                "4711",
                "0815"])
        dataFrame = SevereColumnAdder.addSevereColumn(dataFrame)
        internationalLotTableFactory = InternationalLotTableFactory(dataFrame)
        
        # When
        batchCodeTable = internationalLotTableFactory.createBatchCodeTableByCountry('non existing country')

        # Then
        assert_frame_equal(
            batchCodeTable,
            TestHelper.createDataFrame(
                columns = ['Adverse Reaction Reports', 'Deaths', 'Disabilities', 'Life Threatening Illnesses', 'Company', 'Severe reports', 'Lethality'],
                data = [  ],
                index = pd.Index([], name = 'VAX_LOT')),
            check_dtype = False)


In [ ]:
unittest.main(argv = [''], verbosity = 2, exit = False)

In [ ]:
def getVaersForYear(year):
    return getVaersForYears([year])

def getVaersForYears(years):
    def addCountryColumn(dataFrame):
        dataFrame['COUNTRY'] = 'United States'
        return dataFrame

    return _getVaers(
        _getVaersDescrReader().readVaersDescrsForYears(years),
        addCountryColumn)

def getNonDomesticVaers():
    return _getVaers(
        [_getVaersDescrReader().readNonDomesticVaersDescr()],
        CountryColumnAdder.addCountryColumn)

def _getVaersDescrReader():
    return VaersDescrReader(dataDir = "VAERS")

def _getVaers(vaersDescrs, addCountryColumn):
    dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(vaersDescrs)
    dataFrame = addCountryColumn(dataFrame)
    DataFrameNormalizer.normalize(dataFrame)
    dataFrame = SevereColumnAdder.addSevereColumn(dataFrame)
    return dataFrame
    

In [ ]:
vaers = getVaersForYears([2020, 2021, 2022])
vaers

In [ ]:
nonDomesticVaers = getNonDomesticVaers()
nonDomesticVaers

In [ ]:
internationalVaers = pd.concat([vaers, nonDomesticVaers])
internationalVaers

In [ ]:
def createAndSaveBatchCodeTableForCountry(createBatchCodeTableForCountry, country, minADRsForLethality = None):
    batchCodeTable = createBatchCodeTableForCountry(country)
    batchCodeTable.index.set_names("Batch", inplace = True)
    if minADRsForLethality is not None:
        batchCodeTable.loc[batchCodeTable['Adverse Reaction Reports'] < minADRsForLethality, ['Severe reports', 'Lethality']] = [np.nan, np.nan]
    IOUtils.saveDataFrame(batchCodeTable, '../docs/data/' + country)
    display(country + ":", batchCodeTable)

def createAndSaveBatchCodeTablesForCountries(createBatchCodeTableForCountry, countries, minADRsForLethality = None):
    for country in countries:
        createAndSaveBatchCodeTableForCountry(createBatchCodeTableForCountry, country, minADRsForLethality)

In [ ]:
def printCountryOptions(countries):
    for country in countries:
        printCountryOption(country)

def printCountryOption(country):
    print('<option value="{country}">{country}</option>'.format(country = country))

In [ ]:
countries = sorted(internationalVaers['COUNTRY'].unique())
printCountryOptions(countries)

In [ ]:
minADRsForLethality = 100
internationalLotTableFactory = InternationalLotTableFactory(internationalVaers)

createAndSaveBatchCodeTablesForCountries(
    createBatchCodeTableForCountry = lambda country: internationalLotTableFactory.createBatchCodeTableByCountry(country),
    countries = countries,
    minADRsForLethality = minADRsForLethality)

createAndSaveBatchCodeTableForCountry(
    createBatchCodeTableForCountry = lambda country: internationalLotTableFactory.createGlobalBatchCodeTable(),
    country = 'Global',
    minADRsForLethality = minADRsForLethality)
